# Analyzing Edit Activity of Zurich

In [ ]:
from wikidata.client import Client
import requests
import json
from io import StringIO

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Zurich

In [ ]:
zurich = 'Q72'
canton_zurich = 'Q11943'

client = Client()
print(client.get(zurich, load=True))
print(client.get(canton_zurich, load=True))

## Get the Complete Edit Activity

In [ ]:
def edit_hist(item):
    fields = ['timestamp', 'user', 'userid', 'comment', 'size']
    field_str = '%7C'.join(fields)
    limit = '500'
    query_template = 'https://www.wikidata.org/w/api.php?action=query&format=json&prop=revisions&titles={item}&rvprop={fields}&rvslots=main&rvlimit={limit}'
    query = query_template.format(
        item=item, fields=field_str, limit=limit)

    curr_hist_dict = requests.get(query).json()

    res_str = 'UserID,User,Timestamp,Size,Comment\n'

    while True:
        for page in curr_hist_dict['query']['pages'].values():
            for revision in page['revisions']:
                res_str += str(revision['userid']) + ','
                res_str += revision['user'] + ','
                res_str += revision['timestamp'] + ','
                res_str += str(revision['size']) + ','
                res_str += '"' + revision['comment'] + '"'+ '\n'

        try:
            continue_field = curr_hist_dict['continue']['rvcontinue']
            new_query = query + '&rvcontinue=' + continue_field
            curr_hist_dict = requests.get(new_query).json()
        except KeyError:
            break
            
    return pd.read_csv(StringIO(res_str))

## Get Visit History For the Last 60 Days

In [ ]:
def visit_hist(item):
    query_template = 'https://www.wikidata.org/w/api.php?action=query&format=json&prop=pageviews&titles={item}'
    query = query_template.format(item=item)

    visit_dict = requests.get(query).json()

    res_str = 'Date,VisitCount\n'
    
    for page in visit_dict['query']['pages'].values():
        for date, count in page['pageviews'].items():
            res_str += date + ',' + str(count) + '\n'
            
    return pd.read_csv(StringIO(res_str))

## Analysis

In [28]:
visit_hist(canton_zurich)

,Date,VisitCount
0,2018-07-09,0
1,2018-07-10,2
2,2018-07-11,1
3,2018-07-12,1
4,2018-07-13,None
5,2018-07-14,0
6,2018-07-15,0
7,2018-07-16,None
8,2018-07-17,1
9,2018-07-18,0
